# Experiment: Space-Colonisation Perforated Moss Pole

**Goal.** Grow branch-like tunnels on a cylindrical surface using a 2D space-colonisation algorithm on the unwrapped *(θ, z)* domain, then carve those tunnels through a hollow shell.

**Specs**
- Outer diameter: **50 mm**
- Wall thickness: **2 mm**
- Height: **200 mm**
- Hollow: yes
- Method: space-colonisation on unwrapped cylinder → rasterise branches → cut through wall
- Structural guards:
  - Keep only the **largest 3D connected component**
  - Reserve **no-cut ribs** at 0°, 90°, 180°, 270° by default
  - Tunable minimum strut via tunnel radius

**Outputs**
- `moss_pole_space_colonisation.stl` (and `.3mf` if supported)

**Tuning**
- Increase `attractor_count` for denser branching.
- Adjust `influence_dist_mm`, `kill_dist_mm`, and `step_mm` to change morphology.
- `tunnel_radius_mm` controls hole width; keep ≥ 0.8 mm for FDM, ≥ 1.2-2.0 mm for clay/ceramic.
- Voxel resolution set by `voxel_mm` (default **0.5 mm**).



In [3]:
# igl availability check — do not auto-build; print clear remediation steps
import os, sys
try:
    import igl
    print('igl is available in this Python environment.')
except Exception:
    print('igl not importable in this kernel (ModuleNotFoundError).')
    # If conda is present on PATH, it still requires shell integration to use `conda activate`.
    conda_sh = '/opt/conda/etc/profile.d/conda.sh'
    if os.path.exists(conda_sh):
        print('\nDetected conda installation at /opt/conda.')
        print('If you created the env libigl_env, run these in a terminal (not chained with &&):')
        print(f'  source {conda_sh}')
        print('  conda activate libigl_env')
        print('  conda install -c conda-forge libigl pyigl -y')
    else:
        print('\nConda not detected at /opt/conda. If you need Miniconda, run in a terminal:')
        print('  wget -O ~/miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh')
        print('  bash ~/miniconda.sh -b -p $HOME/miniconda')
        print('  source $HOME/miniconda/etc/profile.d/conda.sh')
        print('  conda install -c conda-forge libigl pyigl -y')
    print('\nImportant: install/activate the env in the shell that launches the notebook or restart the Jupyter kernel after activation.')

igl not importable in this kernel (ModuleNotFoundError).

Detected conda installation at /opt/conda.
If you created the env libigl_env, run these in a terminal (not chained with &&):
  source /opt/conda/etc/profile.d/conda.sh
  conda activate libigl_env
  conda install -c conda-forge libigl pyigl -y

Important: install/activate the env in the shell that launches the notebook or restart the Jupyter kernel after activation.


In [3]:
# Space-Colonisation decorative moss pole on a cylindrical shell
# Dependencies: numpy, scipy, scikit-image, (optional) trimesh
# pip install numpy scipy scikit-image trimesh

import numpy as np
from scipy.ndimage import label, distance_transform_edt
from skimage import measure

try:
    import trimesh
except Exception:
    trimesh = None

# ----------------------- Parameters -----------------------
outer_d_mm   = 50.0
wall_mm      = 2.0
height_mm    = 200.0
voxel_mm     = 0.1                 # 3D voxel size

# Cylindrical UV discretisation (auto from geometry; can override)
theta_cells  = None                # None -> auto from circumference / voxel_mm
z_cells      = None                # None -> auto from height / voxel_mm

# Space-colonisation controls (2D on UV)
rng_seed           = 7
attractor_count    = 3000       # more -> denser branches
influence_dist_mm  = 16.0          # attractors influence nearby nodes
kill_dist_mm       = 3.5           # remove attractor if a node comes this close
step_mm            = 1.8           # growth step per iteration
node_spacing_mm    = 1.6           # min spacing between nodes
max_iters          = 2000

# Tunnel and structure
tunnel_radius_mm   = 1.5          # radius of each perforation hole (smaller)
perforation_spacing_mm = 32.0      # spacing between perforations along veins (larger)

rib_count          = 6             # number of reserved no-cut ribs around θ
rib_width_deg      = 10.0          # angular width for each rib
keep_largest_cc    = True          # remove floating parts
print_stats        = True

# ----------------------- Derived geometry -----------------------
r_out = outer_d_mm * 0.5
r_in  = r_out - wall_mm
assert r_in > 0, "Wall thickness too large."

circum_mm = 2.0 * np.pi * r_out
if theta_cells is None:
    theta_cells = max(128, int(np.round(circum_mm / voxel_mm)))
if z_cells is None:
    z_cells = max(128, int(np.round(height_mm / voxel_mm)))

# Pixel pitch on UV
px_theta_mm = circum_mm / theta_cells
px_z_mm     = height_mm  / z_cells

# ----------------------- UV helpers -----------------------
rng = np.random.default_rng(rng_seed)

def angle_to_col(theta_rad):
    # θ in [0, 2π) -> [0, theta_cells-1]
    return np.mod((theta_rad / (2.0*np.pi)) * theta_cells, theta_cells-1e-6).astype(np.int32)

def z_to_row(z_mm):
    # z in [-H/2, +H/2] -> [0, z_cells-1]
    t = (z_mm + 0.5*height_mm) / height_mm
    t = np.clip(t, 0.0, 1.0)
    return np.floor(t * (z_cells - 1)).astype(np.int32)

# ----------------------- No-cut ribs mask -----------------------
rib_mask = np.zeros((z_cells, theta_cells), dtype=bool)
if rib_count > 0 and rib_width_deg > 0:
    rib_centers = np.linspace(0.0, 2*np.pi, rib_count, endpoint=False)
    halfw = np.deg2rad(rib_width_deg) * 0.5
    theta_vals = (np.arange(theta_cells) + 0.5) / theta_cells * 2*np.pi
    for c in rib_centers:
        # shortest angular distance
        d = np.abs(np.arctan2(np.sin(theta_vals - c), np.cos(theta_vals - c)))
        cols = np.where(d <= halfw)[0]
        if cols.size:
            rib_mask[:, cols] = True

# ----------------------- Attractors -----------------------
# Uniform over UV, avoid ribs to bias growth between ribs
A = []
tries = 0
while len(A) < attractor_count and tries < attractor_count * 10:
    tries += 1
    col = rng.integers(0, theta_cells)
    row = rng.integers(0, z_cells)
    if rib_mask[row, col]:
        continue
    A.append([col + 0.5, row + 0.5])  # center of pixel
A = np.array(A, dtype=np.float32) if A else np.zeros((0,2), dtype=np.float32)

# ----------------------- Initial nodes (stems) -----------------------
# Start a few stems near the base and top to ensure vertical connectivity
stem_cols = np.linspace(0, theta_cells, num=8, endpoint=False) + 0.5
nodes = []
parents = []

base_row = 2.0                      # near z minimum
top_row  = z_cells - 2.0

for c in stem_cols:
    nodes.append([c, base_row])
    parents.append(-1)
    nodes.append([c, top_row])
    parents.append(-1)

nodes = np.array(nodes, dtype=np.float32)
parents = np.array(parents, dtype=np.int32)

# ----------------------- Space-colonisation loop -----------------------
def uv_dist_mm(p, q):
    # anisotropic metric with wrap in θ
    dtheta_px = np.abs(p[0] - q[0])
    dtheta_px = min(dtheta_px, theta_cells - dtheta_px)  # wrap
    dz_px = p[1] - q[1]
    return np.hypot(dtheta_px * px_theta_mm, dz_px * px_z_mm)

infl = influence_dist_mm
kill = kill_dist_mm
step = step_mm
min_node_spacing = node_spacing_mm

attractors = A.copy()
active_mask = np.ones(len(attractors), dtype=bool)

# Spatial bins for nodes to accelerate nearest checks (simple grid)
bin_sz_theta = max(8, int(np.round(infl / px_theta_mm)))
bin_sz_z     = max(8, int(np.round(infl / px_z_mm)))

def grow_once(nodes, parents, attractors, active_mask):
    if not active_mask.any():
        return nodes, parents, attractors, active_mask, False

    # Assign attractors to nearest node if within influence radius
    nz_idx = np.where(active_mask)[0]
    if nz_idx.size == 0:
        return nodes, parents, attractors, active_mask, False

    acc_dirs = [np.zeros(2, dtype=np.float32) for _ in range(len(nodes))]
    touched  = [0]*len(nodes)

    # Kill attractors that are too close to any node
    keep_active = active_mask.copy()
    for ai in nz_idx:
        a = attractors[ai]
        # brute force nearest; acceptable at this scale
        dists = np.array([uv_dist_mm(a, n) for n in nodes], dtype=np.float32)
        j = int(dists.argmin())
        d = float(dists[j])
        if d < kill:
            keep_active[ai] = False
        elif d <= infl:
            # direction from node to attractor in UV with angular wrap
            v = np.array([a[0]-nodes[j][0], a[1]-nodes[j][1]], dtype=np.float32)
            # wrap θ to shortest path
            if v[0] >  0.5*theta_cells: v[0] -= theta_cells
            if v[0] < -0.5*theta_cells: v[0] += theta_cells
            # scale axes to mm before normalisation
            v_mm = np.array([v[0]*px_theta_mm, v[1]*px_z_mm], dtype=np.float32)
            nrm = np.linalg.norm(v_mm) + 1e-8
            acc_dirs[j] += v / (nrm)     # back in pixel space, direction only
            touched[j]  += 1

    if not keep_active.any():
        # nothing left to attract to
        return nodes, parents, attractors, keep_active, False

    # Grow new nodes
    new_nodes = []
    new_parents = []
    for j, cnt in enumerate(touched):
        if cnt == 0:
            continue
        d = acc_dirs[j]
        d_norm = np.linalg.norm(d) + 1e-8
        dir_uv = d / d_norm
        # step in pixels proportional to mm step
        step_pixels_theta = step / px_theta_mm
        step_pixels_z     = step / px_z_mm
        step_pixels = np.array([dir_uv[0]*step_pixels_theta, dir_uv[1]*step_pixels_z], dtype=np.float32)
        p = nodes[j] + step_pixels
        # wrap θ
        p[0] = np.mod(p[0], theta_cells)
        # clamp z inside domain
        p[1] = np.clip(p[1], 0.5, z_cells - 1.5)
        # spacing check
        if nodes.shape[0] < 2:
            ok = True
        else:
            dmin = np.min([uv_dist_mm(p, q) for q in nodes])
            ok = dmin >= min_node_spacing * 0.8
        if ok:
            new_nodes.append(p)
            new_parents.append(j)

    if not new_nodes:
        return nodes, parents, attractors, keep_active, False

    nodes = np.vstack([nodes, np.array(new_nodes, dtype=np.float32)])
    parents = np.hstack([parents, np.array(new_parents, dtype=np.int32)])
    return nodes, parents, attractors, keep_active, True

iters = 0
while iters < max_iters:
    nodes, parents, attractors, active_mask, progressed = grow_once(nodes, parents, attractors, active_mask)
    iters += 1
    if not progressed:
        break

if print_stats:
    print(f"Colonisation iterations: {iters}, nodes: {len(nodes)}, remaining attractors: {active_mask.sum()}")

# ----------------------- Rasterise branches into UV mask -----------------------
perforation_mask = np.zeros((z_cells, theta_cells), dtype=bool)

def draw_perforations_along_segment(p0, p1):
    # Sample points along the segment and mark small circular regions at each
    p0 = np.array(p0); p1 = np.array(p1)
    dθ = p1[0] - p0[0]
    if dθ >  0.5*theta_cells: p1[0] -= theta_cells
    if dθ < -0.5*theta_cells: p1[0] += theta_cells
    seg = p1 - p0
    L_mm = uv_dist_mm(p0, p1)
    n = max(2, int(np.ceil(L_mm / perforation_spacing_mm)))
    ts = np.linspace(0.0, 1.0, n)
    for t in ts:
        q = p0 + t * seg
        q[0] = np.mod(q[0], theta_cells)
        c = int(np.clip(np.floor(q[0]), 0, theta_cells-1))
        r = int(np.clip(np.floor(q[1]), 0, z_cells-1))
        # Mark a circular region of radius tunnel_radius_mm at (r, c)
        rr, cc = np.ogrid[:z_cells, :theta_cells]
        dist_mm = np.hypot((rr - r) * px_z_mm, np.minimum(np.abs(cc - c), theta_cells - np.abs(cc - c)) * px_theta_mm)
        perforation_mask[dist_mm <= tunnel_radius_mm] = True

# draw perforations along all edges
for i, parent in enumerate(parents):
    if parent >= 0:
        draw_perforations_along_segment(nodes[parent], nodes[i])

# Respect ribs: do not cut inside rib_mask
perforation_mask &= ~rib_mask

# ----------------------- Map UV mask to 3D shell and carve -----------------------
# Build 3D grid
pad = 2 * voxel_mm
xmin, xmax = -(r_out + pad), (r_out + pad)
ymin, ymax = -(r_out + pad), (r_out + pad)
zmin, zmax = -(height_mm * 0.5 + pad), (height_mm * 0.5 + pad)

nx = int(np.ceil((xmax - xmin) / voxel_mm))
ny = int(np.ceil((ymax - ymin) / voxel_mm))
nz = int(np.ceil((zmax - zmin) / voxel_mm))

x = np.linspace(xmin + 0.5*voxel_mm, xmax - 0.5*voxel_mm, nx, dtype=np.float32)
y = np.linspace(ymin + 0.5*voxel_mm, ymax - 0.5*voxel_mm, ny, dtype=np.float32)
z = np.linspace(zmin + 0.5*voxel_mm, zmax - 0.5*voxel_mm, nz, dtype=np.float32)
Z, Y, X = np.meshgrid(z, y, x, indexing='ij')

R = np.sqrt(X**2 + Y**2)
Theta = np.arctan2(Y, X)
Theta = np.where(Theta < 0, Theta + 2*np.pi, Theta)

shell = (R >= r_in) & (R <= r_out) & (np.abs(Z) <= (height_mm * 0.5))

theta_idx = np.floor(Theta / (2.0*np.pi) * theta_cells).astype(np.int32)
theta_idx = np.clip(theta_idx, 0, theta_cells - 1)
z_norm = (Z - (-height_mm*0.5)) / height_mm
z_idx  = np.floor(np.clip(z_norm, 0.0, 1.0) * (z_cells - 1)).astype(np.int32)

carve_uv = perforation_mask[z_idx, theta_idx]
carved = shell & (~carve_uv)     # remove where perforation mask is True

# ----------------------- Connectivity filter -----------------------
def largest_component_only(binary_vol):
    structure = np.ones((3,3,3), dtype=np.uint8)
    labeled, n = label(binary_vol, structure=structure)
    if n <= 1:
        return binary_vol, n
    counts = np.bincount(labeled.ravel())
    counts[0] = 0
    keep = counts.argmax()
    return (labeled == keep), n

if keep_largest_cc:
    carved_cc, ncomp = largest_component_only(carved)
else:
    carved_cc, ncomp = carved, 1

if print_stats:
    solid_voxels = int(carved_cc.sum())
    total_voxels = int(shell.sum())
    open_fraction = 1.0 - solid_voxels / max(1, total_voxels)
    print(f"3D components before filter: {ncomp}")
    print(f"Voxels kept: {solid_voxels} / {total_voxels}  (open fraction ≈ {open_fraction:.2%})")

# ----------------------- Mesh extraction and export -----------------------
verts, faces, normals, _ = measure.marching_cubes(
    carved_cc.astype(np.float32),
    level=0.5,
    spacing=(voxel_mm, voxel_mm, voxel_mm)
)

# shift to world coords
verts[:, 0] += zmin
verts[:, 1] += ymin
verts[:, 2] += xmin

if trimesh is not None:
    mesh = trimesh.Trimesh(vertices=verts[:, [2,1,0]], faces=faces, process=True)
    stl_path = "moss_pole_space_colonisation.stl"
    mesh.export(stl_path)
    print(f"Saved: {stl_path}")

    try:
        three_mf_path = "moss_pole_space_colonisation.3mf"
        mesh.export(three_mf_path, file_type='3mf')
        print(f"Saved: {three_mf_path}")
    except Exception:
        print("3MF export not available. STL saved.")
else:
    print("trimesh not installed. Mesh extracted but not exported.")

# ----------------------- Notes -----------------------
# • To increase structural safety, widen ribs (rib_width_deg) or raise rib_count.
# • For finer detail, reduce voxel_mm and raise theta_cells/z_cells accordingly.
# • For more open lace, increase attractor_count or tunnel_radius_mm.
# • For straighter "veins", increase step_mm and reduce influence_dist_mm.
#

Colonisation iterations: 60, nodes: 2195, remaining attractors: 6


KeyboardInterrupt: 

# Experiment: Space-Colonisation Moss Pole — Triangle Mesh Workflow

**Objective**  
Create a structurally sound, decorative moss pole by generating branch-like perforations directly on a **triangulated hollow cylinder**, avoiding voxelisation.  
This method keeps smooth curvature, clean edges, and smaller file sizes.

---

## **Method Overview**

### 1. Base Shell
- Outer diameter: **50 mm**
- Wall thickness: **2 mm**
- Height: **200 mm**
- Construct as a triangulated mesh (inner and outer cylinders) and boolean-subtract inner from outer.

### 2. Pattern Generation
- Unwrap the cylinder to a 2-D *(θ, z)* grid.
- Run a **space-colonisation** growth algorithm in UV space to simulate branching patterns.
- Filter short or low-degree segments (ensure ≥ 3 connections).

### 3. Tube Sweep
- Map UV polylines back to 3-D cylindrical coordinates.  
  \( x = r cos θ,\; y = r sin θ,\; z = z \)
- Sweep a circular cross-section (radius ≈ 1.2 mm) along each branch to create tubular meshes representing perforations.

### 4. Boolean Subtraction
- Boolean difference: `shell − union(branch_tubes)`
- Use a **triangle-based boolean kernel** (e.g. Blender "Exact", CGAL, or libigl).
- Apply remesh modifier to unify edge length (≈ 1 mm) and ensure watertightness.

### 5. Structural Safeguards
- Preserve **no-cut ribs** at 0°, 90°, 180°, 270°.
- Fillet tube ends to reduce stress concentration.
- Optionally run connected-components and retain only the main body.

### 6. Export
- Output:  
  - `moss_pole_space_colonisation_tri.stl`  
  - optional `moss_pole_space_colonisation_tri.3mf`

---

## **Implementation Options**

| Environment | Tools | Notes |
|--------------|-------|-------|
| **Blender Script** | `bpy`, Boolean(Exact), Remesh(Voxel) | Most stable GUI pipeline; simple to iterate visually. |
| **libigl / CGAL (Python)** | `pyigl`, `igl.copyleft.boolean` | Fully scriptable, precise triangle booleans, good for notebooks. |
| **MeshMixer / Meshlab** | Manual proof-of-concept | For visual debugging of exported shells. |

---

## **Tuning**

| Parameter | Typical | Effect |
|------------|----------|--------|
| `tunnel_radius_mm` | 1.2 - 2.0 | Hole size |
| `attractor_count` | 1000 - 3000 | Density of branches |
| `step_mm` | 1.5 - 3.0 | Branch length per iteration |
| `rib_width_deg` | 8 - 12° | Stiffness control |
| `remesh_edge_len` | 0.8 - 1.2 mm | Output smoothness |

---

**Recommended next step:**  
Use the Blender or libigl code cell below this markdown to generate and subtract the branch tubes from the hollow cylinder mesh, producing a printable, organically perforated pole.


Solution — step-by-step
1. Replace the placeholder with the complete mesh-based implementation
2. Use no dependencies beyond pip-installable packages
3. Handle both shell generation and tunnel sweeping
4. Include helper functions for mesh operations

### space_colonisation_perforated_moss-pole.ipynb

Replace the placeholder cell with a complete mesh-based implementation.



In [5]:
import numpy as np
from scipy.spatial.transform import Rotation

def make_cylinder(r, h, n_sides=32, closed=True):
    """Create a triangulated cylinder mesh."""
    # Vertices: rings of points
    theta = np.linspace(0, 2*np.pi, n_sides, endpoint=False)
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    verts = []
    
    if closed:
        # Bottom center
        verts.append([0, 0, -h/2])
    # Bottom rim
    verts.extend(np.column_stack([x, y, np.full_like(x, -h/2)]))
    # Top rim
    verts.extend(np.column_stack([x, y, np.full_like(x, h/2)]))
    if closed:
        # Top center
        verts.append([0, 0, h/2])
    verts = np.array(verts, dtype=np.float32)
    
    faces = []
    if closed:
        # Bottom cap
        center = 0
        first_rim = 1
        for i in range(n_sides):
            i1 = first_rim + (i + 1) % n_sides
            faces.append([center, first_rim + i, i1])
    
    # Side faces
    offset = 1 if closed else 0
    for i in range(n_sides):
        i1 = (i + 1) % n_sides
        # Bottom and top vertices
        v0 = offset + i
        v1 = offset + i1
        v2 = offset + i1 + n_sides
        v3 = offset + i + n_sides
        faces.extend([[v0, v1, v2], [v2, v3, v0]])
    
    if closed:
        # Top cap
        center = len(verts) - 1
        last_rim = center - n_sides
        for i in range(n_sides):
            i1 = (i + 1) % n_sides
            faces.append([center, last_rim + i1, last_rim + i])
    
    return np.array(verts), np.array(faces)

def sweep_polyline(points, radius=1.0, n_sides=16):
    """Sweep a circle along a 3D polyline to create a tubular mesh."""
    if len(points) < 2:
        return None, None
    
    # Generate frames using parallel transport
    tangents = []
    normals = []
    binormals = []
    
    # Initial frame (align with world axes where possible)
    t = points[1] - points[0]
    t = t / np.linalg.norm(t)
    if abs(np.dot(t, [0,0,1])) > 0.999:
        n = np.array([1,0,0])
    else:
        n = np.cross([0,0,1], t)
        n = n / np.linalg.norm(n)
    b = np.cross(t, n)
    
    tangents.append(t)
    normals.append(n)
    binormals.append(b)
    
    # Transport frame along curve
    for i in range(1, len(points)-1):
        t_prev = t
        t = points[i+1] - points[i]
        t = t / np.linalg.norm(t)
        
        # Rotate previous frame
        angle = np.arccos(np.clip(np.dot(t_prev, t), -1.0, 1.0))
        if angle > 1e-6:
            axis = np.cross(t_prev, t)
            axis = axis / np.linalg.norm(axis)
            R = Rotation.from_rotvec(angle * axis)
            n = R.apply(n)
            b = R.apply(b)
        
        tangents.append(t)
        normals.append(n)
        binormals.append(b)
    
    # Last frame
    tangents.append(t)
    normals.append(n)
    binormals.append(b)
    
    # Generate vertices ring by ring
    theta = np.linspace(0, 2*np.pi, n_sides, endpoint=False)
    cos_theta = radius * np.cos(theta)
    sin_theta = radius * np.sin(theta)
    
    verts = []
    for i, center in enumerate(points):
        # Apply frame to circle points
        circle = (np.outer(cos_theta, normals[i]) + 
                 np.outer(sin_theta, binormals[i]))
        verts.extend(circle + center)
    
    # Generate faces between consecutive rings
    faces = []
    for i in range(len(points)-1):
        for j in range(n_sides):
            j1 = (j + 1) % n_sides
            v0 = i * n_sides + j
            v1 = i * n_sides + j1
            v2 = (i+1) * n_sides + j1
            v3 = (i+1) * n_sides + j
            faces.extend([[v0,v1,v2], [v2,v3,v0]])
    
    return np.array(verts), np.array(faces)

def connect_branches(nodes, parents):
    """Convert growth pattern to 3D polylines."""
    lines = []
    for i, parent in enumerate(parents):
        if parent >= 0:  # Skip roots (-1)
            p0 = nodes[parent]
            p1 = nodes[i]
            
            # Convert UV coordinates to 3D
            theta0 = p0[0] / theta_cells * 2*np.pi
            z0 = (p0[1] / z_cells - 0.5) * height_mm
            x0 = r_out * np.cos(theta0)
            y0 = r_out * np.sin(theta0)
            
            theta1 = p1[0] / theta_cells * 2*np.pi
            z1 = (p1[1] / z_cells - 0.5) * height_mm
            x1 = r_out * np.cos(theta1)
            y1 = r_out * np.sin(theta1)
            
            # Add branch
            lines.append(np.array([[x0,y0,z0], [x1,y1,z1]]))
    return lines

try:
    # Check for igl early
    import igl
    
    # Generate base shell
    n_sides = 64
    V_outer, F_outer = make_cylinder(r=r_out, h=height_mm, n_sides=n_sides)
    V_inner, F_inner = make_cylinder(r=r_in, h=height_mm, n_sides=n_sides)
    
    # Shell = outer - inner
    V_shell, F_shell, _ = igl.mesh_boolean(V_outer, F_outer, V_inner, F_inner, 
                                         'minus')
    
    # Convert growth pattern to branch tubes
    branch_lines = connect_branches(nodes, parents)
    tubes = []
    for line in branch_lines:
        V, F = sweep_polyline(line, radius=tunnel_radius_mm, n_sides=16)
        if V is not None:
            tubes.append((V, F))
    
    # Union all tubes then subtract from shell
    if tubes:
        V_tubes, F_tubes = tubes[0]
        for V, F in tubes[1:]:
            V_tubes, F_tubes, _ = igl.mesh_boolean(V_tubes, F_tubes, V, F, 
                                                'union')
        
        # Final boolean: shell - tubes
        V_final, F_final, _ = igl.mesh_boolean(V_shell, F_shell, V_tubes, F_tubes,
                                           'minus')
        
        # Optional: keep largest component only
        if keep_largest_cc:
            C = igl.connected_components(F_final)[0]
            keep = (C == np.argmax(np.bincount(C)))
            V_clean, F_clean = igl.remove_unreferenced(V_final, F_final[keep])
        else:
            V_clean, F_clean = V_final, F_final
        
        # Write result
        igl.write_triangle_mesh("moss_pole_mesh.stl", V_clean, F_clean)
        print(f"Saved mesh version to moss_pole_mesh.stl")
    else:
        print("No branches to process.")

except ImportError:
    print("igl not available. Install with: pip install igl")
    print("Note: If pip install fails, you may need to:")
    print("1. Install build dependencies: apt install cmake build-essential")
    print("2. Build from source: pip install git+https://github.com/libigl/libigl-python-bindings")

igl not available. Install with: pip install igl
Note: If pip install fails, you may need to:
1. Install build dependencies: apt install cmake build-essential
2. Build from source: pip install git+https://github.com/libigl/libigl-python-bindings
